In [84]:
def loadStrongScaling(system):
    exp = "strong_scaling"
    for exp_type in EXP_TYPES:
        for mode in MODES:
            file_name = "./" + system + "/" + exp + "/results/" + exp_type + "/" + exp + "_" + mode + ".txt"
            with open(file_name, "r") as data:
                for line in data:
                    if exp in line or "keycount" in line or "tests" in line:
                        continue

                    line_info = line.strip("\n").split(",")

                    keycount = int(line_info[0])
                    gpucount = int(line_info[1])

                    if line_info[2] == "oom":
                        continue

                    time = float(line_info[2])

                    results[exp][exp_type][mode][keycount].append((gpucount, time))
                
        
def loadWeakScaling(system):
    exp = "weak_scaling"
    for exp_type in EXP_TYPES:
        for mode in MODES:
            file_name = "./" + system + "/" + exp + "/results/" + exp_type + "/" + exp + "_" + mode + ".txt"
            with open(file_name, "r") as data:
                for line in data:
                    if exp in line or "keycount" in line or "tests" in line:
                        continue

                    line_info = line.strip("\n").split(",")

                    keycount = int(line_info[0])
                    gpucount = int(line_info[1])
                    
                    key_per_dev = int(keycount / gpucount)

                    if line_info[2] == "oom":
                        continue

                    time = float(line_info[2])

                    results[exp][exp_type][mode][key_per_dev].append((gpucount, time))
                
        
def loadDuplicateKeys(system):
    exp = "duplicate_keys"
    for exp_type in EXP_TYPES:
        for mode in MODES:
            file_name = "./" + system + "/" + exp + "/results/" + exp_type + "/" + exp + "_" + mode + ".txt"
            with open(file_name, "r") as data:
                for line in data:
                    if exp in line or "keycount" in line or "tests" in line:
                        continue

                    line_info = line.strip("\n").split(",")

                    keycount = int(line_info[0])
                    tablesize = int(line_info[1])
                    gpucount = int(line_info[2])
                    
                    if line_info[3] == "oom":
                        continue

                    time = float(line_info[2])

                    results[exp][exp_type][mode][keycount].append((tablesize, time))
                
        

In [85]:
for system in SYSTEMS:
    loadStrongScaling(system)

In [86]:
for system in SYSTEMS:
    loadWeakScaling(system)

In [87]:
for system in SYSTEMS:
    loadDuplicateKeys(system)

In [88]:
print(len(results["duplicate_keys"]["build"]["noindex_nomanaged"].items()))
for k, v in results["duplicate_keys"]["build"]["noindex_nomanaged"].items():
    print(k, v)
print("here3?")

1
67108864 [(2097152, 6.0), (4194304, 6.0), (8388608, 6.0), (16777216, 6.0), (33554432, 6.0), (67108864, 6.0), (134217728, 6.0), (268435456, 6.0)]
here3?


In [91]:
def get_cmap(n, name='hsv'):
    '''Returns a function that maps each index in 0, 1, ..., n-1 to a distinct 
    RGB color; the keyword argument name must be a standard mpl colormap name.'''
    return plt.cm.get_cmap(name, n)

def plotThroughput(system, exp, exp_type, mode):
    fig, axes = plt.subplots(nrows=1,sharex=True)
    # fig.set_size_inches(5,3)

    plot_data = results[exp][exp_type][mode]
    
    cmap = get_cmap(len(plot_data.items()))
    for i, data in enumerate(plot_data.items()):
        keyc = data[0]
        data_points = data[1]
        x = [i[0] for i in data_points]
        y = [i[1] for i in data_points]
        
        for j in range(len(y)):
            if exp == "weak_scaling":
                y[j] = (keyc * x[j]) / y[j]
            else:
                y[j] = keyc / y[j]
        
        label = "2^" + str(int(np.log(keyc) / np.log(2)))
        axes.plot(x, y, label=label,
                  color=cmap(i),
                  marker=fmarkers[i])
        
    axes.set_ylim(ymin=0)
    
    #axes.plot(threads, threads, label="Ideal",color=plt.rcParams['axes.color_cycle'][4])
    #axes.set_yticks(np.arange(0, 5.0, 0.5))

    plt.title(system + " " + exp + " " + exp_type + " " + mode, pad=20)
    plt.legend(loc='upper left', ncol=1, fancybox=True, shadow=True, fontsize=11)
    
    axes.set_ylabel("Keys / sec.",fontsize=8)    
    axes.set_xlabel("GPU Count",fontsize=8)    
    #plt.xticks(fontsize=8)
    #plt.yticks(fontsize=8)
    
    # plt.savefig(mytitle + ".pdf", format="pdf");
    plt.show()
     
def plotSpeedup(system, exp, exp_type, mode):
    fig, axes = plt.subplots(nrows=1,sharex=True)
    # fig.set_size_inches(5,3)

    plot_data = results[exp][exp_type][mode]
    
    cmap = get_cmap(len(plot_data.items()))
    for i, data in enumerate(plot_data.items()):
        keyc = data[0]
        data_points = data[1]
        x = [i[0] for i in data_points]
        y = [i[1] for i in data_points]
        
        # compute keys / sec throughput (higher is better)
        for j in range(len(y)):
            if exp == "weak_scaling":
                y[j] = (keyc * x[j]) / y[j]
            else:
                y[j] = keyc / y[j]
            
        # compute speedup of throughput
        for j in range(len(y)):
            if j == 0:
                continue
            y[j] = y[j] / y[0]
        y[0] = 1.0
        
        label = "2^" + str(int(np.log(keyc) / np.log(2)))
        axes.plot(x, y, label=label,
                  color=cmap(i),
                  marker=fmarkers[i])
        
    axes.set_ylim(ymin=0)
    
    #axes.plot(threads, threads, label="Ideal",color=plt.rcParams['axes.color_cycle'][4])
    #axes.set_yticks(np.arange(0, 5.0, 0.5))

    plt.title(system + " " + exp + " " + exp_type + " " + mode, pad=20)
    plt.legend(loc='upper left', ncol=1, fancybox=True, shadow=True, fontsize=11)
    
    axes.set_ylabel("Speedup",fontsize=8)    
    axes.set_xlabel("GPU Count",fontsize=8)    
    #plt.xticks(fontsize=8)
    #plt.yticks(fontsize=8)
    
    # plt.savefig(mytitle + ".pdf", format="pdf");
    plt.show()
     

In [ ]:
print("Unmanaged Memory Plots")
for sys in SYSTEMS:
        for exp_type in EXP_TYPES:
            for exp in EXPERIMENTS:
                if exp == "duplicate_keys":
                    continue
                plotThroughput(system, exp, exp_type, "noindex_nomanaged")
            # plotSpeedup(system, exp, exp_type, "index_nomanaged")
            
print("Managed Memory Plots")
for sys in SYSTEMS:
        for exp_type in EXP_TYPES:
            for exp in EXPERIMENTS:
                if exp == "duplicate_keys":
                    continue
                plotThroughput(system, exp, exp_type, "noindex_managed")
            # plotSpeedup(system, exp, exp_type, "index_managed")